# Charity Shop Optimise Floor Plan

In this notebook we will calculate the optimal number of bays for each category for each month based on data for the month's previous year as well as all available previous years. We will also obtain the average actual number of bays, the average actual weekly revenue of the shop and the predicted revenue when using the suggested optimal number of bays. These findings will then be written to an excel file ready to be used when needed.

In [1]:
from scipy.optimize import linprog
import pandas as pd
import numpy as np
df_sm = pd.read_excel("Charity Shop.xlsx", sheet_name = "Space Management", index_col = None)

In [3]:
df_sm.head()

,date week commencing,category,sub category,Dept £,Dept %,No. items sold,Average selling price £,No. of bays,% of space,Average sale per bay £,Day,Month,Year,Modified Month,Modified Year
0,20/07/2025,clothing,Ladies Tops,451.37,12.11,72,6.269028,2.0,8.33,225.685,20,7,2025,7,2025
1,20/07/2025,clothing,Ladies Knitwear,54.75,1.47,9,6.083333,0.5,2.08,109.500,20,7,2025,7,2025
2,20/07/2025,clothing,Ladies Skirts,85.81,2.30,12,7.150833,0.5,2.08,171.620,20,7,2025,7,2025
3,20/07/2025,clothing,Ladies Trousers,186.06,4.99,29,6.415862,1.0,4.17,186.060,20,7,2025,7,2025
4,20/07/2025,clothing,Ladies Dresses,140.25,3.76,15,9.350000,3.0,12.50,46.750,20,7,2025,7,2025


In [5]:
df_sm.shape

(4633, 15)

We remove the category EOL clothing (end of line clothing) since this was only used in the most recent 16 weeks out of 171 (excluding 3 in january 2024) and the revenue and quantity sold for this category is very small so will have very little difference to our final result.

In [7]:
df_sm = df_sm[df_sm['sub category'] != 'Eol Clothing']

The bought in goods categories have fixed values for each month and we store them below in lists (lenght of 12 for 12 months). BIG cards wrap have a fixed value of 0.5 each month so a list is not needed for this category.

In [11]:
xmas_cards = [1, 0.1, 0, 0, 0, 0, 0, 0.5, 0.5, 2, 2, 2]
big_gifts = [1.5, 1.3, 1.25, 1, 1, 1, 1, 1, 1.3, 2, 2, 2]

We have a function below which performs the linear programming optimisation to obtain the values we are looking for. It takes in the parameters optim_time which is the time period for what we use as our training data. This can either be the previous year available for the month or all previous years' worth of data available for the month. The other parameter is the month for which we are optimising.

In [13]:
def optimise_floor_plan(optim_time, month):
    xmas_cards_amount = xmas_cards[month - 1] #get xmas cards number of bays for selected month
    big_gifts_amount = big_gifts[month - 1] #get big gifts number of bays for selected month
    if optim_time == 'all': #all years worth of data for the month
        df = df_sm[df_sm['Modified Month'] == month] #create dataframe for the selected month, for all years available
    else:
        if month in {1,2,3,4,5,6,7}: #since our data goes up to july 2025, january to july have their most recent year as 2025
            df = df_sm[(df_sm['Modified Month'] == month) & (df_sm['Modified Year'] == 2025)]
        else:
            df = df_sm[(df_sm['Modified Month'] == month) & (df_sm['Modified Year'] == 2024)] #the rest of the months have their most recent year as 2024
    avg_num_bays = np.round(df[df['sub category'] == 'Donated And Big Total']['No. of bays'].mean(),1) #get avaerage total number of bays for this month
    #we will use this amount as the total number of bays for the optimised model. This number can change each month due to the limitation of the /
    #company's software only allowing you to input a minimum of 0.5 bays per category, even if there's only 1 item for sale for it
    df_subcats = df[df['category'] != 'total'] #get rid of the sub categories with a category value of total since these are not items for sale 
    pivot = pd.pivot_table(df_subcats, values = ['Average sale per bay £', 'No. items sold', 'Dept %', 'Dept £', '% of space', 'No. of bays'], index = ['sub category'], aggfunc = 'mean')
    #create pivot table to work out average sale per bay per sub cat, average quantity of items sold, average revenue per sub cat, average number of bays per sub cat, /
    #all weekly average
    actual_num_bays = pivot['No. of bays'].to_numpy() #get array containing actual number of bays for each sub cat
    actual_rev = df[df['sub category'] == 'Donated And Big Total']['Dept £'].mean() #get average actual revenue for the shop for this month in the selected time period
    c = pivot['Average sale per bay £'].to_numpy(dtype = float) #our objective function to maximise, we are finding the otimal number of bays for each \
    #sub category which will maximise the revenue in the shop for each week in the selected month and time period
    c = np.multiply(c, -1) #since we are maximising we have to multiply the objective function by -1

    bounds = (0,3) #each category must be between 0 and 3, some have individual constraints too as can be seen below
    #equality constraints
    A_eq = [[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], #all bays sums to avg_num_bays
           [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #big cards have to be 0.5
           [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #big gifts have to be the amount from big_gifts array
           [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #xmas cards have to be the amount from xmas_cards array
           [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #jewellery is fixed at 0.5 due to space requirements
           ]
    b_eq = [avg_num_bays, 0.5, big_gifts_amount, xmas_cards_amount, 0.5] #the fiexed value for the equalities stated above

    #min and max value constraints for sub category number of bay values
    A_ub = [
           [0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #books min 1.5 
           [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #books max 2 
           [0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #home min 2.5
           [0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], #kids clothing min 1
           [0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0], #kids non clothing min 1
           [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], #kids non clothing max 2
           [0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0], #ladies accs min 0.75
           [0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0], #ladies coats min 0.25
           [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], #ladies coats max 1
           [0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0], #ladies dresses min 0.5
           [0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0], #ladies knitwear min 0.5
           [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], #ladies knitwear max 1
           [0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0], #ladies shoes min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0], #ladies shoes max 1.5
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0], #ladies skirts min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0], #ladies tops min 1
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0], #ladies trousers min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0], #ladies trousers max 1
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0], #media min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0], #media max 0.75
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0], #mens bottoms min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0], #mens bottoms max 0.75
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0], #mens coats jackets and suits min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0], #mens coats jackets and suits max 0.75
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0], #mens shoes accs min 0.25
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0], #mens shoes accs max 0.75
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1], #mens tops min 0.5
           [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1] #mens tops max 2
           ]
    #the min and max values
    b_ub = [-1.5, 2, -2.5, -1, -1, 2, -0.75, -0.25, 1, -0.5, -0.25, 1.5, -0.25, 1.5, -0.5, -1, -0.25, 1, -0.25, 0.75, -0.25, 0.75, -0.25, 0.75, -0.25, 0.75, -0.5, 2]

    res = linprog(c = c, A_eq = A_eq, b_eq = b_eq, A_ub = A_ub, b_ub = b_ub, bounds = bounds, options={"disp": True}) #result of the linear programming optimisation
    percent_diff_rev = np.round(100*((abs(res.fun) - actual_rev) / actual_rev),2) #percentage difference between observed average weekly revenue and predicted optimal revenue
    sub_cats = pivot.index #get sub cat values
    return sub_cats, np.round(actual_num_bays, 2), res.x, np.round(actual_rev, 2), np.round(abs(res.fun), 2), percent_diff_rev
    #return actual observed average number of bays for each sub cat, predicted optimal number of bays for each subcat, observed average actual weekly revenue for the shop, \
    #predicted weekly revenue when using optimal number of bays, percentage difference from two revenue values

In [15]:
#two lists which will be converted into dataframes once filled with the data
optim_df_list = []
optim_rev_list = []

#loop through each month for the time period using all years and add the values obtained from the optimise_floor_plan function to optim_df_list
for month in range(1,13):
    sub_cats, actual_num_bays, optim_num_bays, actual_rev, optim_rev, percent_diff_rev = optimise_floor_plan('all', month)
    month_rev = []
    month_rev.append(actual_rev)
    month_rev.append(optim_rev)
    month_rev.append(percent_diff_rev)
    month_rev.append('All Years')
    month_rev.append(month)
    optim_rev_list.append(month_rev)
    for i in range(0, len(sub_cats)):
        sub_cat_list = []
        sub_cat_list.append(sub_cats[i])
        sub_cat_list.append(actual_num_bays[i])
        sub_cat_list.append(optim_num_bays[i])
        sub_cat_list.append('All Years')
        sub_cat_list.append(month)
        optim_df_list.append(sub_cat_list)

#loop through each month for the time period using each month's previous year and add the values obtained from the optimise_floor_plan function to optim_rev_list
for month in range(1,13):
    sub_cats, actual_num_bays, optim_num_bays, actual_rev, optim_rev, percent_diff_rev = optimise_floor_plan('prev', month)
    month_rev = []
    month_rev.append(actual_rev)
    month_rev.append(optim_rev)
    month_rev.append(percent_diff_rev)
    month_rev.append('Previous Year')
    month_rev.append(month)
    optim_rev_list.append(month_rev)
    for i in range(0, len(sub_cats)):
        sub_cat_list = []
        sub_cat_list.append(sub_cats[i])
        sub_cat_list.append(actual_num_bays[i])
        sub_cat_list.append(optim_num_bays[i])
        sub_cat_list.append('Previous Year')
        sub_cat_list.append(month)
        optim_df_list.append(sub_cat_list)

In [17]:
#convert lists to dataframes
optim_bays_df = pd.DataFrame(optim_df_list, columns=['Sub Category', 'Average Actual Num Bays', 'Optim Num Bays', 'Time Period', 'Month'])
optim_rev_df = pd.DataFrame(optim_rev_list, columns=['Actual Revenue', 'Predicted Optimal Revenue', 'Percentage Difference', 'Time Period', 'Month'])

In [19]:
#export dataframes to excel files ready to be visualised in power bi
optim_bays_df.to_excel("optim bays.xlsx", index = False)
optim_rev_df.to_excel("optim rev.xlsx", index = False)